In [13]:
import os

import weave

os.environ["WF_TRACE_SERVER_URL"] = "http://127.0.01:6345"

client = weave.init("action_test_6")

Logged in as Weights & Biases user: timssweeney.
View Weave data at https://wandb.ai/timssweeney/action_test_6/weave


In [2]:
# from weave.trace import autopatch, weave_init
# from weave.trace_server import clickhouse_trace_server_batched

# ch_server = clickhouse_trace_server_batched.ClickHouseTraceServer.from_env()
# inited_client = weave_init.InitializedClient(client)
# autopatch.autopatch()


In [14]:
from openai import OpenAI


@weave.op
def extract_name(user_input: str) -> str:
    openai_client = OpenAI()
    if "Tim" in user_input:
        return "i don't know!"
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Extract the name from the user input. If there is no name, return an empty string.",
            },
            {"role": "user", "content": user_input},
        ],
        temperature=0.0,
        max_tokens=64,
        top_p=1,
    )
    return response.choices[0].message.content


calls = []
names = ["Scott", "Adrian", "Tim", "Jeff", "Shawn"]
for name in names:
    res, call = extract_name.call(f"My name is {name}.")
    calls.append(call)

print(calls)

🍩 https://wandb.ai/timssweeney/action_test_6/r/call/0192b7ad-c69e-7fa3-8429-dd762263f7b0
🍩 https://wandb.ai/timssweeney/action_test_6/r/call/0192b7ad-c84e-7f62-b2d9-087bcbe7fbd0
🍩 https://wandb.ai/timssweeney/action_test_6/r/call/0192b7ad-ca28-7093-a385-8de632d4563d
🍩 https://wandb.ai/timssweeney/action_test_6/r/call/0192b7ad-ca2f-7993-82e7-5f700cede96c
🍩 https://wandb.ai/timssweeney/action_test_6/r/call/0192b7ad-cbc0-7593-8d83-5c4024a63105
[Call(_op_name=<Future at 0x2b98acfa0 state=finished returned str>, trace_id='0192b7ad-c69e-7fa3-8429-dd8f2c5b82db', project_id='timssweeney/action_test_6', parent_id=None, inputs={'user_input': 'My name is Scott.'}, id='0192b7ad-c69e-7fa3-8429-dd762263f7b0', output='Scott', exception=None, summary={'usage': {'gpt-3.5-turbo-0125': {'requests': 1, 'completion_tokens': 1, 'prompt_tokens': 35, 'total_tokens': 36, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}}}, display_name=None, attributes=Attribu

In [15]:
from weave.collection_objects import action_objects
from weave.trace.weave_client import get_ref
from weave.trace_server.interface.collections import action_collection

action = action_objects.ActionWithConfig(
    name="is_name_extracted",
    action=action_collection._BuiltinAction(
        name="openai_completion",
    ),
    config={
        "model": "gpt-4o-mini",
        "system_prompt": "Given the following prompt and response, determine if the name was extracted correctly.",
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "is_name_extracted",
                "schema": {
                    "type": "object",
                    "properties": {"is_extracted": {"type": "boolean"}},
                    "required": ["is_extracted"],
                    "additionalProperties": False,
                },
                "strict": True,
            },
        },
    },
)
mapping = action_objects.ActionOpMapping(
    name="extract_name-is_name_extracted5",
    action=action,
    op_name=get_ref(extract_name).name,
    op_digest=get_ref(extract_name).digest,
    input_mapping={
        "prompt": "inputs.user_input",
        "response": "output",
    },
)

In [16]:
from weave.trace_server import trace_server_interface as tsi

res = client.server.execute_batch_action(
    req=tsi.ExecuteBatchActionReq(
        project_id=client._project_id(),
        call_ids=[c.id for c in calls[:1]],
        mapping=mapping,
    )
)